In [ ]:
print(__doc__)
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import check_random_state
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import mutual_info_score

from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

pd.options.display.float_format = "{:.1f}".format

Automatically created module for IPython interactive environment


In [ ]:
def retrieve_data_recid():

    """ This function is used for retrieving dataset COMPAS and split data entries into labeled (training, testing) and unlabeled data (validation) 
    Prediction task is to determine whether a person will recidive after first prosecution"""

    """ Binary classification
    =================  ======================
    samples total      5875
    Dimensionality     9(Features)+1(Bias)
    Features           real
    Classes            2
    =================  ======================

    Source
    ----------
    How We Analyzed the COMPAS Recidivism Algorithm, by Jeff Larson, Surya Mattu, Lauren Kirchner and Julia Angwin, May 23, 2016
    https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm
    https://github.com/propublica/compas-analysis
    
    Parameters
    ----------
    none

    Returns
    -------
    X_labeled: training data, ndarray, shape (10, 9)
    y_labeled: training target, ndarray, shape (10, ) 
    b_labeled: bias attribute of training set, ndarray, shape (10, )

    X_unlabeled: sample pool data, ndarray, shape (4397, 9)
    y_unlabeled: sample pool target, ndarray, shape (4397, )
    b_unlabeled: bias attribute of sample pool set, ndarray, shape (4397, )

    X_test: testing data, ndarray, shape (1468, 9)
    y_test: testing target, ndarray, shape (1468, )
    b_test: bias attribute of testing set, ndarray, shape (1468, )
    """
    
    attributes = ['MarriageStatus','age','juv_fel_count', 'juv_misd_count', 'juv_other_count','priors_count', 'days_b_screening_arrest','c_days_from_compas','c_charge_degree']
    bias = 'race'
    target = 'two_year_recid'

    # np.random.seed(42)
    data = pd.read_csv("https://raw.githubusercontent.com/WenxuanHuang/Active-Learning-Performance-Benchmarking/main/RecidivismData_Normalized.csv", sep=',')
    data_col = data.columns
    df = data[(data[bias]==2)|(data[bias]==3)].copy().values
    # print(df.shape)

    kf = KFold(n_splits=4)
    for train_index, test_index in kf.split(df):
        train, test = df[train_index], df[test_index]
        # print("Size of X_train_full, X_test:", train.shape, test.shape)

    df_train = pd.DataFrame(data=train, columns=data_col)
    df_test = pd.DataFrame(data=test, columns=data_col)

    labeled = df_train.groupby(target, group_keys=False).apply(lambda x: x.sample(n=5)) # ten sample in total labeled initially
    # labeled = df_train.groupby(target, group_keys=False).apply(lambda x: x.sample(n=5, random_state=42)) # with a random state for stable output
    df_X_labeled = labeled[attributes]
    df_y_labeled = labeled[target]
    X_labeled = df_X_labeled.values
    y_labeled = df_y_labeled.values.astype('int64')
    b_labeled = labeled[bias].values-2 #degrade bias into binary options
    (row_size, col_size) = X_labeled.shape 
    # print(X_labeled.shape)

    unlabeled = df_train.drop(df_X_labeled.index)
    df_X_unlabeled = unlabeled[attributes]
    df_y_unlabeled = unlabeled[target]
    X_unlabeled = df_X_unlabeled.values
    y_unlabeled = df_y_unlabeled.values.astype('int64')
    b_unlabeled = unlabeled[bias].values-2
    # print(X_unlabeled.shape)

    X_test = df_test[attributes].values
    y_test = df_test[target].values
    y_test=y_test.astype('int')
    b_test = df_test[bias].values-2
    # print(X_test.shape)

    X_fair_est = X_unlabeled
    y_fair_est = y_unlabeled
    b_fair_est = b_unlabeled
    
    return (X_labeled, y_labeled, b_labeled, row_size, col_size, X_unlabeled, y_unlabeled, b_unlabeled, X_test, y_test, b_test, X_fair_est, y_fair_est, b_fair_est)

In [ ]:
class BaseModel(object):

    def __init__(self):
        pass
    def fit_predict(self):
        pass

class LogModel(BaseModel):

    def fit_predict(self, X_labeled, y_labeled, X_test, y_test):
        self.classifier = LogisticRegression(
            solver='liblinear'
            )
        self.classifier.fit(X_labeled, y_labeled)
        features_weight= self.classifier.coef_.T
        # self.y_test_predicted = self.classifier.predict(X_test)
        # self.y_unlabeled_predicted = self.classifier.predict(X_unlabeled)
        self.y_test_score = self.classifier.score(X_test, y_test)
        return (X_labeled, X_test, self.y_test_score, features_weight)

In [ ]:
class TrainModel:

    def __init__(self, model_object):        
        self.accuracies = []
        self.model_object = model_object()        

    def print_model_type(self):
        print (self.model_object.model_type)

    def train(self, X_labeled, y_labeled, X_test, y_test):
        (X_labeled, X_test, self.y_test_score, features_weight) = \
            self.model_object.fit_predict(X_labeled, y_labeled, X_test, y_test)
        return (X_labeled, X_test, features_weight)

    def get_test_accuracy(self, i):
        classif_rate = self.y_test_score * 100
        self.accuracies.append(classif_rate)               
        print('--------------------------------')
        print('Iteration:',i)
        # print("Accuracy rate is %f " % (classif_rate))

In [ ]:
def random_selection(probas_val, step):
    # random_state = check_random_state(63)
    selection = np.random.choice(probas_val.shape[0], step, replace=False)
    return selection

# def entropy_selection(probas_val, step):
#     e = (-probas_val * np.log2(probas_val)).sum(axis=1)
#     selection = (np.argsort(e)[::-1])[:step]
#     return selection

In [ ]:
def normalizer(e_loss, f_loss):
    e_loss = np.reshape(e_loss, (1,len(e_loss)))
    f_loss = np.reshape(f_loss, (1,len(f_loss)))
    e_scaled = preprocessing.normalize(e_loss)
    # e_scaled=((e_loss-e_loss.min())/(e_loss.max()-e_loss.min()))
    f_scaled = preprocessing.normalize(f_loss)
    # f_scaled=((f_loss-f_loss.min())/(f_loss.max()-f_loss.min()))
    e_scaled = e_scaled.flatten()
    f_scaled = f_scaled.flatten()
    return (e_scaled, f_scaled)

In [ ]:
def log_loss(probas_val):
    
    eps = np.finfo(probas_val.dtype).eps
    probas_val = np.clip(probas_val, eps, 1 - eps)
    e_loss = (-probas_val * np.log2(probas_val)).sum(axis=1)

    return e_loss

In [ ]:
""" This function is used for calculating fairness loss, aka the unfairness of a logistic regression model. The fairness measurement is used to provide loss values that helps determine the fluctuation of unfairness change with each iteration that adds samples to the labeled dataset. There are many different methods for unfairness measurement, however, five different meausures are coded and only one of them will be used for unfairness measurement in the correlation-based sampling. """

""" Formulas

Mutual Information: MI(B,Y') = \sum_{i = 1}^{|B|}\sum_{j = 1}^{|Y'|}\frac{|B_{i} \cap Y'_{j}|}{N} log\frac{N|B_{i} \cap Y'_{j}|}{|B_{i}||Y'_{j}|}
Statistical Parity: SP(B,Y') = |P(Y' = 1|B = a) - P(Y' = 1|B = b)| \quad \forall a,b \in B
Equal Opportunity: EOP(B,Y') = |P(Y' = 1|B = a, Y = 1) - P(Y' = 1|B = b, Y = 1)| \quad \forall a,b \in B
Equalized Odds: EOD(B,Y') = |P(Y' = 1|B = a, Y = y) - P(Y' = 1|B = b, Y = y)| \quad \forall a,b \in B, \quad \forall 0,1 \in y
Conditional Use Accuracy Equality: CUAE(B,Y') = |P(Y' = y|B = a, Y = y) - P(Y' = y|B = b, Y = y)| \quad \forall a,b \in B, \quad \forall 0,1 \in y
Disparate Impact: DI(B,Y') = |P(Y' = 1|B = a) / P(Y' = 1|B = b)| \quad \forall a,b \in B

Source
----------

separation \ Equal opportunity - Hardt, Price, Srebro (2016)
separation \ Equalized adds - Hardt, Price, Srebro (2016)
independence \ Mutual information - Cover, T.M.; Thomas, J.A. (1991). Elements of Information Theory (Wiley ed.)
sufficiency \ Conditional Use Accuracy Equality - Richard Berk et al., (2017) Fairness in Criminal Justice Risk Assessments: The State of the Art

Parameters
----------
X_fair_est: testing data, ndarray
y_fair_est: testing target, ndarray
b_fair_est: bias attribute of testing set, ndarray

Returns
-------
f_loss: fairness loss, int
"""

def mut_inf(X_fair_est, b_fair_est, classifier):
    
    y_fair_pred = classifier.predict(X_fair_est)

    f_loss=mutual_info_score(b_fair_est, y_fair_pred)
    f_loss=abs(f_loss)
    
    return f_loss

def stats_parity(X_fair_est, b_fair_est, classifier):
    
    y_fair_pred = classifier.predict(X_fair_est)

    b0p1=X_fair_est[(b_fair_est==0)&(y_fair_pred==1)].shape[0]
    b0=X_fair_est[(b_fair_est==0)].shape[0]
    b1p1=X_fair_est[(b_fair_est==1)&(y_fair_pred==1)].shape[0]
    b1=X_fair_est[(b_fair_est==1)].shape[0]

    f_loss=(b0p1/b0)-(b1p1/b1)
    
    return f_loss

def eqops(X_fair_est, y_fair_est, b_fair_est, classifier):
    
    y_fair_pred = classifier.predict(X_fair_est)

    b0y1p1=X_fair_est[(b_fair_est==0)&(y_fair_pred==1)&(y_fair_est==1)].shape[0]
    b0y1=X_fair_est[(b_fair_est==0)&(y_fair_est==1)].shape[0]
    b1y1p1=X_fair_est[(b_fair_est==1)&(y_fair_pred==1)&(y_fair_est==1)].shape[0]
    b1y1=X_fair_est[(b_fair_est==1)&(y_fair_est==1)].shape[0]

    f_loss=(b0y1p1/b0y1)-(b1y1p1/b1y1)
    
    return f_loss

def eqods(X_fair_est, y_fair_est, b_fair_est, classifier):
    
    y_fair_pred = classifier.predict(X_fair_est)

    b0y0p1=X_fair_est[(b_fair_est==0)&(y_fair_est==0)&(y_fair_pred==1)].shape[0]
    b0y0=X_fair_est[(b_fair_est==0)&(y_fair_est==0)].shape[0]
    b1y0p1=X_fair_est[(b_fair_est==1)&(y_fair_est==0)&(y_fair_pred==1)].shape[0]
    b1y0=X_fair_est[(b_fair_est==1)&(y_fair_est==0)].shape[0]

    b0y1p1=X_fair_est[(b_fair_est==0)&(y_fair_est==1)&(y_fair_pred==1)].shape[0]
    b0y1=X_fair_est[(b_fair_est==0)&(y_fair_est==1)].shape[0]
    b1y1p1=X_fair_est[(b_fair_est==1)&(y_fair_est==1)&(y_fair_pred==1)].shape[0]
    b1y1=X_fair_est[(b_fair_est==1)&(y_fair_est==1)].shape[0]

    fpr_loss=abs((b0y0p1/b0y0)-(b1y0p1/b1y0))
    tpr_loss=abs((b0y1p1/b0y1)-(b1y1p1/b1y1))


    f_loss = (fpr_loss+tpr_loss)/2 # justification of half in terms of fpr_loss UNION tpr_loss
    
    return f_loss 

def cuae(X_fair_est, y_fair_est, b_fair_est, classifier):
    
    y_fair_pred = classifier.predict(X_fair_est)

    b0y0p0=X_fair_est[(b_fair_est==0)&(y_fair_est==0)&(y_fair_pred==0)].shape[0]
    b0y0=X_fair_est[(b_fair_est==0)&(y_fair_est==0)].shape[0]
    b1y0p0=X_fair_est[(b_fair_est==1)&(y_fair_est==0)&(y_fair_pred==0)].shape[0]
    b1y0=X_fair_est[(b_fair_est==1)&(y_fair_est==0)].shape[0]

    b0y1p1=X_fair_est[(b_fair_est==0)&(y_fair_est==1)&(y_fair_pred==1)].shape[0]
    b0y1=X_fair_est[(b_fair_est==0)&(y_fair_est==1)].shape[0]
    b1y1p1=X_fair_est[(b_fair_est==1)&(y_fair_est==1)&(y_fair_pred==1)].shape[0]
    b1y1=X_fair_est[(b_fair_est==1)&(y_fair_est==1)].shape[0]

    tnr_loss=abs((b0y0p0/b0y0)-(b1y0p0/b1y0))
    tpr_loss=abs((b0y1p1/b0y1)-(b1y1p1/b1y1))

    f_loss = (tnr_loss+tpr_loss)/2
    
    return f_loss 

def disp_impt(X_fair_est, b_fair_est, classifier):
    
    y_fair_pred = classifier.predict(X_fair_est)

    b0p1=X_fair_est[(b_fair_est==0)&(y_fair_pred==1)].shape[0]
    b0=X_fair_est[(b_fair_est==0)].shape[0]
    b1p1=X_fair_est[(b_fair_est==1)&(y_fair_pred==1)].shape[0]
    b1=X_fair_est[(b_fair_est==1)].shape[0]

    f_loss=(b0p1/b0)/(b1p1/b1)
    f_loss = f_loss-1
    return f_loss
# selecting fairness criteria
def fair_measure(X_fair_est, y_fair_est, b_fair_est, classifier=None, criteria=0):
    if criteria == 'mutual_information':
        return mut_inf(X_fair_est, b_fair_est, classifier)
    elif criteria == 'equal_opportunity':
        return eqops(X_fair_est, y_fair_est, b_fair_est, classifier)
    elif criteria == 'statistical_parity':
        return stats_parity(X_fair_est, b_fair_est, classifier)
    elif criteria == 'equalized_odds':
        return eqods(X_fair_est, y_fair_est, b_fair_est, classifier)
    elif criteria == "disparate_impact":
        return disp_impt(X_fair_est, b_fair_est, classifier)
    elif criteria == "conditional_use_accuracy_equality":
        return cuae(X_fair_est, y_fair_est, b_fair_est, classifier)

In [ ]:
def fair_loss_corr(X_data, prob_expected, row_size, y_labeled_sum, yhatb_corr, Xy_labeled_sum, X_labeled_sum, features_weight, Xb_corr, Xy_corr, Xy_std):

    print("Correlation between X and S:", Xb_corr)
    # print("Correlation between y' and S:", yhatb_corr)
    # print("d:prob_expected:", prob_expected.shape, prob_expected) output: shape(2,1), probabilities of y' =0 and 1

    fair_loss = 0
    fair_improvement = 0
    row_expected = row_size + 1 
    for i in range(len(prob_expected)): 
        y_expected = (y_labeled_sum + i)/row_expected
        Xy_expected = np.add(Xy_labeled_sum , X_data*i)/row_expected
        X_expected = np.add(X_labeled_sum , X_data)/row_expected
        Xy_corr_expected = (np.subtract(Xy_expected, (X_expected * y_expected)))/Xy_std
        print("Correlation between X and y:", Xy_corr)
        print("Expected correlation between X and y:", Xy_corr_expected)
        fair_improvement = np.dot((abs(features_weight*Xb_corr)).transpose(),(abs(Xy_corr)-abs(Xy_corr_expected)).reshape(Xy_corr_expected.shape[0],1))[0,0] # if expected corr(xy) is reduced, the feature weight is higher
        fair_loss += prob_expected[i]*fair_improvement # fairness improvement for a point, both y=1 and y=0 are added as one
        # print("Shape of fair_improvement and temp[i]",fair_improvement, prob_expected[i])
    return fair_loss

In [ ]:
def unfairness_correlation_sampling(X_unlabeled, classifier, probas_val, step, row_size, y_labeled_sum, yhatb_corr, Xy_labeled_sum, X_labeled_sum, features_weight, Xb_corr, Xy_corr, div, Xy_std):
    # further to be defined, now assume only fairness loss
    # query size used here

    unlabeled_size = len(X_unlabeled)
    f_loss = np.zeros(unlabeled_size)
    
    for j in range(unlabeled_size): 
            proba_temp = classifier.predict_proba(X_unlabeled[j].reshape(1, -1)).reshape(-1,1)
            # first reshape integrate as single entry
            # second reshape transpose as vertical 2D array
            f_loss[j] = fair_loss_corr(X_unlabeled[j],proba_temp, row_size, y_labeled_sum, yhatb_corr, Xy_labeled_sum, X_labeled_sum, features_weight, Xb_corr, Xy_corr, Xy_std)

    e_loss = log_loss(probas_val)

#     print("Debug f_loss before:", f_loss.shape, f_loss)

    e_scaled, f_scaled = normalizer(e_loss, f_loss)
    f_scaled[np.isnan(f_scaled)] = 0

#     print("Debug f_loss after:", f_scaled.shape, f_scaled)

    loss = div*(e_scaled)+(1-div)*f_scaled
    selection = np.argsort(loss)[::-1][:step]
    

    return selection

In [ ]:
def pre_filter(X_data, b_data, y_data, budget): 

    temp_columns = ['MarriageStatus','age','juv_fel_count', 'juv_misd_count', 'juv_other_count','priors_count', 'days_b_screening_arrest','c_days_from_compas','c_charge_degree','race','two_year_recid']
    # np.random.seed(84)

    # print("Debug data:", X_data.shape, b_data.shape, y_data.shape)
    temp_data = np.c_[X_data, b_data, y_data]
    temp_df = pd.DataFrame(data=temp_data, columns=temp_columns)
    candidates_data = temp_df.groupby('race', group_keys=False).apply(lambda x: x.sample(n=math.ceil(budget/2)))
    # candidates_data = temp_df.groupby('race', group_keys=False).apply(lambda x: x.sample(n=math.ceil(budget/2), random_state = 84))
    # print("Debug ceil:", math.ceil(budget/2))
    # print("Debug ceil:", math.ceil(budget/2))
    # print("Debug cand:", candidates)
    candidates_index = candidates_data.index.values
    # print("Debug index:", candidates_index.shape, candidates_index)

    return candidates_index

In [ ]:
def metrics(X_test, y_test, b_test, classifier):
    y_test_pred = classifier.predict(X_test)
    # print("Classification report for classifier %s:\n%s\n" % (classifier, classification_report(y_test, y_test_pred)))
    # print("Confusion matrix:\n%s" % confusion_matrix(y_test, y_test_pred))

    # tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
    # tpr = tp/(tp+fn)
    # tnr = tn/(tn+fp) 
    # fpr = fp/(fp+tn)
    # fnr = fn/(tp+fn)
    # fdr = fp/(tp+fp)

    # fig, ax = plt.subplots()

    # model_displays = {}
    # for i in np.unique(y_test):
    #     model_displays[i] = plot_roc_curve(
    #         classifier, X_test[b_test==i], y_test[b_test==i], ax=ax, name=i)
    # ax.set_title('ROC curve')
    # plt.show() 

In [ ]:
def scatters(X_unlabeled, b_unlabeled, probas_val, uncertain_samples):
    y_plot = probas_val
    fig=plt.figure()
    ax=fig.add_axes([0,0,1,1])
    ax.scatter(X_unlabeled[:,1], y_plot[:,1], c=b_unlabeled, cmap = 'Pastel1')
    ax.scatter(X_unlabeled[:,1][uncertain_samples], y_plot[:,1][uncertain_samples], c=b_unlabeled[uncertain_samples], edgecolors='black', s=100, cmap = 'Pastel1')
    ax.set_xlabel('Age')
    ax.set_ylabel('Score')
    ax.set_title('Population plot')
    y_plot = np.delete(y_plot, uncertain_samples, axis=0)

In [ ]:
def performance_progression(init_index, x_axis, budget, step, ur_fairness, rs_fairness, nonal_fairness, ur_accuracies, rs_accuracies, nonal_accuracies, pplot_file_name,run):
    fig, (ax1, ax2) = plt.subplots(2,figsize=(10,10))
    fig.suptitle('Fairness and accuracy metrics')
    ax1.plot(x_axis, ur_fairness, color='r', label='unfair-active')
    ax1.plot(x_axis, rs_fairness, color='g', label='random-active')
    ax1.plot(x_axis, nonal_fairness, color='b', label='non-active')
    ax1.legend()
    ax2.plot(x_axis, ur_accuracies, color='r', label='unfair-active')
    ax2.plot(x_axis, rs_accuracies, color='g', label='random-active')
    ax2.plot(x_axis, nonal_accuracies, color='b', label='non-active')
    ax2.legend()
    ax1.set_xlabel('Sample size')
    ax1.set_ylabel('Unfairness')
    ax2.set_ylabel('Accuracies')
    plt.savefig(pplot_file_name.format(run), bbox_inches='tight', dpi=200)
    plt.show()

In [ ]:
def trade_off_plot(ur_fairness, rs_fairness, nonal_fairness, ur_accuracies, rs_accuracies, nonal_accuracies, tplot_file_name, smooth_tplot_file_name,run):
    fig = plt.figure(figsize=(10,5))
    ur_index = np.argsort(ur_accuracies)
    rs_index = np.argsort(rs_accuracies)
    nonal_index = np.argsort(nonal_accuracies) 

    plt.plot(np.array(ur_accuracies)[ur_index], np.array(ur_fairness)[ur_index], color='r', label='unfair-active')
    plt.plot(np.array(rs_accuracies)[rs_index], np.array(rs_fairness)[rs_index], color='g', label='random-active')
    plt.plot(np.array(nonal_accuracies)[nonal_index], np.array(nonal_fairness)[nonal_index], color='b', label='non-active')
    plt.legend()
    plt.xlabel('Accuracies')
    plt.ylabel('Unfairness')
    plt.savefig(tplot_file_name.format(run), bbox_inches='tight', dpi=200)
    plt.show()

    ur_smooth = sm.nonparametric.lowess(np.array(ur_fairness)[ur_index], np.array(ur_accuracies)[ur_index], frac = 0.5)
    rs_smooth = sm.nonparametric.lowess(np.array(rs_fairness)[rs_index], np.array(rs_accuracies)[rs_index], frac = 0.5)
    nonal_smooth = sm.nonparametric.lowess(np.array(nonal_fairness)[nonal_index], np.array(nonal_accuracies)[nonal_index], frac = 0.5)

    fig = plt.figure(figsize=(10,5))
    plt.plot(ur_smooth[:, 0], ur_smooth[:, 1], color='r', label='unfair-active')
    plt.plot(rs_smooth[:, 0], rs_smooth[:, 1], color='g', label='random-active')
    plt.plot(nonal_smooth[:, 0], nonal_smooth[:, 1], color='b', label='non-active')
    plt.legend()
    plt.xlabel('Approximate accuracies')
    plt.ylabel('Approximate unfairness')
    plt.savefig(smooth_tplot_file_name.format(run), bbox_inches='tight', dpi=200)
    plt.show()

In [ ]:
""" Correlation-based sample selection:  """

""" Formulas

Mutual Information: MI(B,Y') = \sum_{i = 1}^{|B|}\sum_{j = 1}^{|Y'|}\frac{|B_{i} \cap Y'_{j}|}{N} log\frac{N|B_{i} \cap Y'_{j}|}{|B_{i}||Y'_{j}|}
Statistical Parity: SP(B,Y') = |P(Y' = 1|B = a) - P(Y' = 1|B = b)| \quad \forall a,b \in B
Equal Opportunity: EOP(B,Y') = |P(Y' = 1|B = a, Y = 1) - P(Y' = 1|B = b, Y = 1)| \quad \forall a,b \in B
Equalized Odds: EOD(B,Y') = |P(Y' = 1|B = a, Y = y) - P(Y' = 1|B = b, Y = y)| \quad \forall a,b \in B, \quad \forall 0,1 \in y
Conditional Use Accuracy Equality: CUAE(B,Y') = |P(Y' = y|B = a, Y = y) - P(Y' = y|B = b, Y = y)| \quad \forall a,b \in B, \quad \forall 0,1 \in y
Disparate Impact: DI(B,Y') = |P(Y' = 1|B = a) / P(Y' = 1|B = b)| \quad \forall a,b \in B

Source
----------

separation \ Equal opportunity - Hardt, Price, Srebro (2016)
separation \ Equalized adds - Hardt, Price, Srebro (2016)
independence \ Mutual information - Cover, T.M.; Thomas, J.A. (1991). Elements of Information Theory (Wiley ed.)
sufficiency \ Conditional Use Accuracy Equality - Richard Berk et al., (2017) Fairness in Criminal Justice Risk Assessments: The State of the Art

Parameters
----------
X_fair_est: testing data, ndarray
y_fair_est: testing target, ndarray
b_fair_est: bias attribute of testing set, ndarray

Returns
-------
f_loss: fairness loss, int
"""

class active_learning(object):

    def __init__(self, step, budget, model_object, criteria):
        self.step = step
        self.budget = budget
        self.model_object = model_object
        # self.sample_selection_function = selection_function
        self.criteria = criteria
        
    def run(self, X_labeled, y_labeled, b_labeled, row_size, col_size, X_unlabeled, y_unlabeled, b_unlabeled, X_test, y_test, b_test, X_fair_est, y_fair_est, b_fair_est, sub_option, af_div):
  
        self.clf_model = TrainModel(self.model_object)
        (X_labeled, X_test, features_weight) = self.clf_model.train(X_labeled, y_labeled, X_test, y_test)

        Xb_corr = np.corrcoef(np.concatenate((X_unlabeled, b_unlabeled.reshape(b_unlabeled.shape[0],1)),1).T.astype(float))[0:col_size,-1].reshape(col_size,1) # the correlation of features and sensitive attribute (do not iteratively change)

        """Xb correlation: should it be the correlation of X and b for labeled or unlabeled data?"""

        Xy_labeled_sum = np.zeros(col_size) # 1D array
        X_labeled_sum = np.zeros(col_size) # 1D array
        y_labeled_sum = np.sum(y_labeled) #integer, sum of all labeled y

        for i in range(row_size):
            X_labeled_sum = np.add(X_labeled_sum, X_labeled[i])
            Xy_labeled_sum = np.add(Xy_labeled_sum, X_labeled[i]*y_labeled[i])

        y_mean = 1./row_size*y_labeled_sum
        Xy_corr = (1./row_size*np.subtract(Xy_labeled_sum,X_labeled_sum*y_mean))/(np.std(X_labeled)*np.std(y_labeled)) # (sum of x for y=1 - sum of x with ratio of y=1)/row size
        yhatb_corr = np.dot(Xb_corr.transpose(),features_weight)[0,0] # Correlation between prediction of y and biased feature

        active_iteration = 1
        self.clf_model.get_test_accuracy(active_iteration)
        self.query_size = len(X_labeled)
        fairness = []
        fairness = np.append(fairness, fair_measure(X_test, y_test, b_test, classifier=self.clf_model.model_object.classifier, criteria=self.criteria))
        metrics(X_test, y_test, b_test, self.clf_model.model_object.classifier)


        while self.query_size <= self.budget-self.step:

            active_iteration += 1
            self.query_size += self.step

            probas_val = \
                self.clf_model.model_object.classifier.predict_proba(X_unlabeled)

            Xy_std = np.std(X_labeled)*np.std(y_labeled)

            # yb_corr=np.dot(Xb_corr.transpose(),features_weight)[0,0]
            
            # print("Debug predicted:", y_unlabeled_predicted.shape)
            # print("Debug probas_val:", probas_val.shape)

            # if sub_option == "Legacy_filter":
            #     # post_entropy_index = entropy_selection(probas_val, self.budget*2)
            #     candidates_index = pre_filter(X_unlabeled, b_unlabeled, y_unlabeled, self.budget)
            #     uncertain_samples = unfairness_reduction_sampling(self.query_size, X_unlabeled[candidates_index], X_labeled, y_labeled, self.clf_model.model_object.classifier, X_fair_est, y_fair_est, b_fair_est, probas_val[candidates_index], self.step, self.criteria)

            if sub_option == "Pre_filter":
                candidates_index = pre_filter(X_unlabeled, b_unlabeled, y_unlabeled, self.budget)
                uncertain_samples = unfairness_correlation_sampling(X_unlabeled[candidates_index], self.clf_model.model_object.classifier, probas_val[candidates_index], self.step, row_size, y_labeled_sum, yhatb_corr, Xy_labeled_sum, X_labeled_sum, features_weight, Xb_corr, Xy_corr, af_div, Xy_std)

            elif sub_option == "No_filter": 
                uncertain_samples = unfairness_correlation_sampling(X_unlabeled, self.clf_model.model_object.classifier, probas_val, self.step, row_size, y_labeled_sum, yhatb_corr, Xy_labeled_sum, X_labeled_sum, features_weight, Xb_corr, Xy_corr, af_div, Xy_std)

            elif sub_option == "Filter_only":
                uncertain_samples = random_selection(probas_val, self.step)
            # print("Debug shape of X_unlabeled and loss:", selection)

            # scatters(X_unlabeled, b_unlabeled, probas_val, uncertain_samples)

            X_labeled = np.concatenate((X_labeled, X_unlabeled[uncertain_samples]))
            y_labeled = np.concatenate((y_labeled, y_unlabeled[uncertain_samples]))

            (X_labeled, X_test, features_weight) = self.clf_model.train(X_labeled, y_labeled, X_test, y_test)

            row_size += self.step

            for i in uncertain_samples:

                X_labeled_sum = np.add(X_labeled_sum,X_unlabeled[i])
                # print("Debug before update:", Xy_labeled_sum)
                y_labeled_sum += y_unlabeled[i]; 
                Xy_labeled_sum = np.add(Xy_labeled_sum, (X_unlabeled[i]*y_unlabeled[i]))

            y_expected = 1./row_size*y_labeled_sum
            Xy_corr = (1./row_size*np.subtract(Xy_labeled_sum,X_labeled_sum*y_expected))/(np.std(X_labeled)*np.std(y_labeled))
            yhatb_corr = np.dot(Xb_corr.transpose(),features_weight)[0,0] # update based on features weight change
            # print("Debug Xb corr:", Xb_corr)
            # print("Debug yhatb corr:", yhatb_corr)

            X_unlabeled = np.delete(X_unlabeled, uncertain_samples, axis=0)
            y_unlabeled = np.delete(y_unlabeled, uncertain_samples, axis=0)
            b_unlabeled = np.delete(b_unlabeled, uncertain_samples, axis=0)

            fairness = np.append(fairness, fair_measure(X_test, y_test, b_test, classifier=self.clf_model.model_object.classifier, criteria=self.criteria))
            self.clf_model.get_test_accuracy(active_iteration)

            metrics(X_test, y_test, b_test, self.clf_model.model_object.classifier)

        return self.clf_model.accuracies, fairness

In [ ]:
def non_active_learning(init_index, X_unlabeled, y_unlabeled, X_labeled,y_labeled, X_test, y_test, b_test, budget, step, unfairness_criteria): 

    initial_X_train = X_labeled
    initial_y_train = y_labeled
    nonal_X_train = X_unlabeled
    nonal_y_train = y_unlabeled
    nonal_X_test = X_test
    nonal_y_test = y_test
    nonal_b_test = b_test
    nonal_fairness = []

    nonal_accuracies=[]

    classifier_nonal = LogisticRegression(
            solver='liblinear'
            )

    classifier_nonal.fit(initial_X_train, initial_y_train)
    initial_y_pred = classifier_nonal.predict(nonal_X_test)
    nonal_fairness = np.append(nonal_fairness, fair_measure(nonal_X_test, nonal_y_test, nonal_b_test, classifier=classifier_nonal, criteria=unfairness_criteria))
    nonal_accuracies.append(accuracy_score(nonal_y_test, initial_y_pred)*100)

    for i in np.arange(init_index+step,budget+1,step):
        classifier_nonal.fit(nonal_X_train[:i], nonal_y_train[:i])
        nonal_y_pred = classifier_nonal.predict(nonal_X_test)
        nonal_fairness = np.append(nonal_fairness, fair_measure(nonal_X_test, nonal_y_test, nonal_b_test, classifier=classifier_nonal, criteria=unfairness_criteria))
        nonal_accuracies.append(accuracy_score(nonal_y_test, nonal_y_pred)*100)
        metrics(X_test, y_test, b_test, classifier_nonal)

    return nonal_accuracies, nonal_fairness

In [ ]:
def experiment(model,budget,step,criteria,af_div,sub_option=None):
    
    (X_labeled, y_labeled, b_labeled, row_size, col_size, X_unlabeled, y_unlabeled, b_unlabeled, X_test, y_test, b_test, X_fair_est, y_fair_est, b_fair_est) = retrieve_data_recid()
    init_index = len(X_labeled)

    act_alg = active_learning(step, budget, model, criteria)

    ur_accuracies, ur_fairness = act_alg.run(X_labeled, y_labeled, b_labeled, row_size, col_size, X_unlabeled, y_unlabeled, b_unlabeled, X_test, y_test, b_test, X_fair_est, y_fair_est, b_fair_est, sub_option, af_div)

    rs_accuracies, rs_fairness = act_alg.run(X_labeled, y_labeled, b_labeled, row_size, col_size, X_unlabeled, y_unlabeled, b_unlabeled, X_test, y_test, b_test, X_fair_est, y_fair_est, b_fair_est, "Filter_only", af_div)

    nonal_accuracies, nonal_fairness = non_active_learning(init_index, X_unlabeled, y_unlabeled, X_labeled, y_labeled, X_test, y_test, b_test, budget, step, criteria)

    x_axis = np.arange(init_index,budget+1,step)

    return init_index, x_axis, budget, step, ur_fairness, rs_fairness, nonal_fairness, ur_accuracies, rs_accuracies, nonal_accuracies

In [ ]:
def results(init_index, x_axis, budget, step, ur_fairness, rs_fairness, nonal_fairness, ur_accuracies, rs_accuracies, nonal_accuracies, run):

    data = (np.array([x_axis, ur_accuracies, ur_fairness, rs_accuracies, rs_fairness, nonal_accuracies, nonal_fairness])).T

    log_file_name = "result_log/{run}.Result.csv"
    pplot_file_name = "result_log/{:03}.Perf_Plot.png"
    tplot_file_name = "result_log/{:03}.Trdoff_Plot.png"
    smooth_tplot_file_name = "result_log/{:03}.Smooth.Trdoff_Plot.png"
    np.savetxt(log_file_name.format(run = run), data, delimiter=',', header="x_axis, ur_accuracies, ur_fairness, rs_accuracies, rs_fairness, nonal_accuracies, nonal_fairness", fmt='%d,%f,%f,%f,%f,%f,%f')

    performance_progression(init_index, x_axis, budget, step, ur_fairness, rs_fairness, nonal_fairness, ur_accuracies, rs_accuracies, nonal_accuracies, pplot_file_name, run)
    trade_off_plot(ur_fairness, rs_fairness, nonal_fairness, ur_accuracies, rs_accuracies, nonal_accuracies, tplot_file_name, smooth_tplot_file_name, run)

In [ ]:
for i in np.arange(10):

    af_div = i/10
    print("Ratio of e_loss/f_loss is",af_div,"/",1-af_div)

    init_index, x_axis, budget, step, ur_fairness, rs_fairness, nonal_fairness, ur_accuracies, rs_accuracies, nonal_accuracies = experiment(LogModel,200,10,"equalized_odds",af_div,"No_filter")
    results(init_index, x_axis, budget, step, ur_fairness, rs_fairness, nonal_fairness, ur_accuracies, rs_accuracies, nonal_accuracies, run = i)

Ratio of e_loss/f_loss is 0.0 / 1.0
--------------------------------
Iteration: 1
Correlation between X and S: [[ 0.15911661]
 [ 0.21997391]
 [-0.07898785]
 [-0.10179434]
 [-0.04955809]
 [-0.17465952]
 [ 0.02652215]
 [-0.01820103]
 [-0.09742302]]
Correlation between X and y: [0.0 0.3140553858065363 0.0011631680955797642 0.0008947446898022555
 0.0006842165220221102 0.009795099773677424 0.014960666428543271
 -0.0001483851911095771 0.0]
Expected correlation between X and y: [0.0 0.28454360024100017 0.001153555136112163 0.0008873501055890137
 0.0006785618400219274 0.009916526632058172 0.012643884388514373
 -0.0001670232866095179 -0.0019225918935202775]
Correlation between X and y: [0.0 0.3140553858065363 0.0011631680955797642 0.0008947446898022555
 0.0006842165220221102 0.009795099773677424 0.014960666428543271
 -0.0001483851911095771 0.0]
Expected correlation between X and y: [0.0 0.2864661921345204 0.0009612959467601357 0.0007394584213241781
 0.0005654682000182728 0.007892745683718961 0.